# Neuromatch Academy: Week 3, Day 1, Tutorial 4 (Bonus)
# Real Neurons: Spike-timing dependent plasticity (STDP)
__Content creators:__ Qinglong Gu, Songtin Li, John Murray, Richard Naud, Arvind Kumar

__Content reviewers:__ Spiros Chavlis, Lorenzo Fontolan, Richard Gao, Matthew Krause

---
# Tutorial Objectives
In this tutorial, we will focus on building a model of a synapse in which its synaptic strength changes as a function of the relative timing (i.e., time difference) between the spikes of the presynaptic and postsynaptic neurons, respectively. This change in the synaptic weight is known as **spike-timing dependent plasticity (STDP)**.

Our goals for this tutorial are to:

- build a model of synapse that show STDP

- study how correlations in input spike trains influence the distribution of synaptic weights

Towards these goals, we will model the presynaptic input as Poisson type spike trains. The postsynaptic neuron will be modeled as an LIF neuron (see Tutorial 1).

Throughout this tutorial, we assume that a single postsynaptic neuron is driven by $N$ presynaptic neurons. That is, there are $N$ synapses, and we will study how their weights depend on the statistics or the input spike trains and their timing with respect to the spikes of the postsynaptic neuron.


---
# Setup

In [ ]:
# Import libraries
import matplotlib.pyplot as plt
import numpy as np
import time

In [ ]:
# @title Figure Settings
import ipywidgets as widgets       # interactive display

%config InlineBackend.figure_format='retina'
# use NMA plot style
plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/nma.mplstyle")
my_layout = widgets.Layout()

In [ ]:
# @title Helper functions


def default_pars_STDP(**kwargs):
  pars = {}

  # typical neuron parameters
  pars['V_th'] = -55.     # spike threshold [mV]
  pars['V_reset'] = -75.  # reset potential [mV]
  pars['tau_m'] = 10.     # membrane time constant [ms]
  pars['V_init'] = -65.   # initial potential [mV]
  pars['V_L'] = -75.      # leak reversal potential [mV]
  pars['tref'] = 2.       # refractory time (ms)

  # STDP parameters
  pars['A_plus'] = 0.008                   # magnitude of LTP
  pars['A_minus'] = pars['A_plus'] * 1.10  # magnitude of LTD
  pars['tau_stdp'] = 20.                   # STDP time constant [ms]

  # simulation parameters
  pars['T'] = 400.  # Total duration of simulation [ms]
  pars['dt'] = .1   # Simulation time step [ms]

  # external parameters if any
  for k in kwargs:
    pars[k] = kwargs[k]

  pars['range_t'] = np.arange(0, pars['T'], pars['dt'])  # Vector of discretized time points [ms]

  return pars


def Poisson_generator(pars, rate, n, myseed=False):
  """Generates poisson trains

  Args:
    pars            : parameter dictionary
    rate            : noise amplitute [Hz]
    n               : number of Poisson trains
    myseed          : random seed. int or boolean

  Returns:
    pre_spike_train : spike train matrix, ith row represents whether
                      there is a spike in ith spike train over time
                      (1 if spike, 0 otherwise)
  """

  # Retrieve simulation parameters
  dt, range_t = pars['dt'], pars['range_t']
  Lt = range_t.size

  # set random seed
  if myseed:
    np.random.seed(seed=myseed)
  else:
    np.random.seed()

  # generate uniformly distributed random variables
  u_rand = np.random.rand(n, Lt)

  # generate Poisson train
  poisson_train = 1. * (u_rand < rate * (dt / 1000.))

  return poisson_train


def my_raster_plot(range_t, spike_train, n):
  """Generates poisson trains

  Args:
    range_t     : time sequence
    spike_train : binary spike trains, with shape (N, Lt)
    n           : number of Poisson trains plot

  Returns:
    Raster_plot of the spike train
  """

  # Find the number of all the spike trains
  N = spike_train.shape[0]

  # n should be smaller than N:
  if n > N:
    print('The number n exceeds the size of spike trains')
    print('The number n is set to be the size of spike trains')
    n = N

  # Raster plot
  i = 0
  while i <= n:
    if spike_train[i, :].sum() > 0.:
      t_sp = range_t[spike_train[i, :] > 0.5]  # spike times
      plt.plot(t_sp, i * np.ones(len(t_sp)), 'k|', ms=10, markeredgewidth=2)
    i += 1
  plt.xlim([range_t[0], range_t[-1]])
  plt.ylim([-0.5, n + 0.5])
  plt.xlabel('Time (ms)')
  plt.ylabel('Neuron ID')


def my_example_P():
  spT = pre_spike_train_ex[pre_spike_train_ex.sum(axis=1) > 0., :]
  plt.figure(figsize=(7, 6))
  plt.subplot(211)
  color_set = ['r', 'b', 'k', 'orange', 'c']
  for i in range(spT.shape[0]):
    t_sp = pars['range_t'][spT[i, :] > 0.5]   # spike times
    plt.plot(t_sp, i*np.ones(len(t_sp)), '|', color=color_set[i],
             ms=10, markeredgewidth=2)
  plt.xlabel('Time (ms)')
  plt.ylabel('Neuron ID')
  plt.xlim(0, 200)

  plt.subplot(212)
  for k in range(5):
    plt.plot(pars['range_t'], P[k, :], color=color_set[k], lw=1.5)
  plt.xlabel('Time (s)')
  plt.ylabel('P(t)')
  plt.xlim(0, 200)

  plt.tight_layout()


def mySTDP_plot(A_plus, A_minus, tau_stdp, time_diff, dW):
  plt.figure()
  plt.plot([-5 * tau_stdp, 5 * tau_stdp], [0, 0], 'k', linestyle=':')
  plt.plot([0, 0], [-A_minus, A_plus], 'k', linestyle=':')

  plt.plot(time_diff[time_diff <= 0], dW[time_diff <= 0], 'ro')
  plt.plot(time_diff[time_diff > 0], dW[time_diff > 0], 'bo')

  plt.xlabel(r't$_{\mathrm{pre}}$ - t$_{\mathrm{post}}$ (ms)')
  plt.ylabel(r'$\Delta$W', fontsize=12)
  plt.title('Biphasic STDP', fontsize=12, fontweight='bold')
  plt.show()

`helper functions`:

*default_pars_STDP*: initialise neuron parameters and external parameters

*Poisson_generator*: generates poisson trains with raster plot - neuron id (yaxis) vs time (xaxis)

*my_example_P*: plot presynaptic trains with neuron id / P(t) (yaxis) vs time (xaxis)

*mySTDP_plot*: plt biphasic STDP with delta (yaxis) and pre-post time difference (xaxis)

---
# Section 1: Spike-timing dependent plasticity (STDP)

In [ ]:
#@title Video 1: STDP
from IPython.display import YouTubeVideo
video = YouTubeVideo(id='luHL-mO5S1w', width=854, height=480, fs=1)
print("Video available at https://youtube.com/watch?v=" + video.id)
video

# Summary of Video 1:

**spike timing and plasticity.**

- When two neurons are connected
the connection strength is
plastic, meaning that it is
malleable, it can be sculpted.
It can be altered.
Well, many experiments have shown that if you stimulate
neurons on both sides of the synapse, you may alter the effect of a spike
arriving on the postsynaptic cell.
- Synaptic plasticity is often described by,
saying 'what fires together wires together'?
How is plasticity
orchestrated when the communication is based on spikes with precise timing ?
A weight is
a metric for the connection strength, when a presynaptic cell fires an action potential it creates a postsynaptic current and
the amplitude of the postsynaptic current is what we called the synaptic weight.
Now if we make some type of plasticity protocol
we make the presynaptic and postsynaptic neuron fire.
This
postsynaptic current when we trigger the presynaptic cell again
might say increase. If it increases 
we call this long term potentiation or
potentiation simply. If it decreases, 
We call that long term depression,
LTD.
- Experimental studies have shown that there are multiple factors that influence whether we see LTP or LTD.
Relative timing is an important one.
When the presynaptic neuron fires an action potential before the postsynaptic neuron,
such as it could have been causing the postsynaptic neuron to fire,
then we see potentiation.
When it's the reverse,
when it's the postsynaptic neuron that fires before,
we typically see
LTD depression.
- Neuromodulation is another very important factor.
No matter what type of electrical activity we have on both sides of the synapse
We can go from LTD to LTP by changing the relative
concentration of
neuromodulators that are deployed at the synapse.
These neuromodulators are
serotonin, acetylcholine and dopamine.
And another factor is the firing pattern. For instance, when we pair a
presynaptic spike with a postsynaptic
single spike, we typically get LTD.
But when we pair presynaptic spike with a postsynaptic burst, we typically get LTP
So by simply changing the firing pattern on the postsynaptic neuron,
we can alter the type of plasticity.
- pair a train of action potential
stimulated and the presynaptic neuron and a train of action potential with a postsynaptic neuron and
we control the relative timing between the pre and the post spikes.
Then we measure the synaptic weight and we see then that when post precedes pre we get
LTD and
that when pre
precedes post
we get LTP.
- our simulations will not take the form of these very regular spikes.
Instead the spike timing will be rather random such as we see in
-vivo when we look at spike trains.
So how should this synaptic strength W evolve? How should we change
this W should we actually look at the nearest neighbors and compute some type of
relative timing or should we actually take triplet interaction ? How should we go about doing this?
- The approach that is typically taken is to use
traces.
Presynaptic traces and postsynaptic traces.
This is thought to reflect that many processes in biology are involving 
sudden change followed by a linear dynamics that that give you the trace.
For instance, a presynaptic trace is thought to be an activation of a metabotropic
receptor
on the postsynaptic
membrane
such that whenever a presynaptic spike arrives,
the trace for that particular neuron, it
increases and then decays with linear dynamics to its stationary point.
We can control the amplitude of that increase with a parameter A+.
Conversely, we have a postsynaptic trace whenever a postsynaptic spike is emitted
there the action potential is back propagating to the location of the synapse
and this leaves a trace;
whenever we have a postsynaptic spike, we have a jump
and then linear dynamics back to a stationary point here. Here we'll choose to jump to be negative and
control this with a parameter A- and we call the trace M.
In
this way
if each
presynaptic spike will change the weight according to the postsynaptic trace
each pre-synaptic spike is
looking at a postsynaptic trace and reads it off here at zero. So no change in weights.
But when there's a presynaptic spike, I have a negative value another negative value. So I decrease the synaptic weight
This can be written here as reading off and the postsynaptic trace at the time of the presynaptic spike.
We make that proportional to the weight
so that we have a multiplicative change.
The postsynaptic spike will also incur a change of the weights and
each time we have a postsynaptic spike, we read off of the presynaptic trace. This way
we really capture the relative timing. In this case whenever the postsynaptic stress spike comes when the
presynaptic trace is elevated, we will increase the weights
and this way we get LTP whenever pre precedes post and we get
LTD whenever post precedes pre.
And we can create this sort of coincidence window where the change in weights
will be LTP whenever pre precedes post, but that
magnitude of the potentiation would depend on the precise timing and
this decay here is controlled by the decay of the
postsynaptic trace of the presynaptic trace and is not a parameter that is chosen to be 20
milliseconds to match experiments.
The amplitude here will be controlled by A+ and the amplitude here of the depression will be controlled by A-.
But we're not finished
Connection strengths cannot keep on increasing forever and
similarly, they cannot decrease and then
become negative,
because neurons do not go from being excitatory to suddenly become inhibitory.
So we need to impose bounds on the dynamics or the synaptic weights
where
whenever we are changing upon a presynaptic spike
instead of increasing
to an infinite bound we will compare the
increase that we wanted to take
With some upper bound here one and take the minimum value of the two such that
we cannot increase the synaptic weight further.
Similarly when a postsynaptic spikes and we would normally
implement some depression,
we will take the maximum value between what we would like to do according to the synaptic traces and zero.
This way we're bound between 0 and 1.

Imagine a thousand excitatory neurons firing random spikes
at a fixed
presynaptic firing rate and
200 inhibitory neurons that are also firing at a fixed presynaptic firing rate.
Only the synapses of the excitatory neurons are plastic are allowed to change and
the postsynaptic neuron
will fire in response to all this activity.
So the presynaptic activity will look fairly random and
if we don't choose the synaptic weights here to be strong enough,
we won't be able to drive the postsynaptic neuron to fire and there will be no pairs
to speak of and there will be no plasticity and the system will just remain silent.
So there's no pre and post coincidence no plasticity and neurons remain silent.
If instead we start the simulation with stronger connections
such that the postsynaptic neuron is active
then we're typically in the case where there's quite a lot of
current flowing through and the postsynaptic neuron is in this mean driven regime and fires quite
regular action potentials and it's not locked to any particular
firings in the presynaptic cells
So we can consider the association between pre and post to be random. So
regular firing means that we're sampling randomly from this coincidence window
so if there's just as much
LTP as there is LTD we should get no net changes in the synaptic weights.
But if we choose that there is a net depression
That allows the system to do some type of
self-organization and it's quite interesting.

The synaptic weight will give net depression. It will reduce the net drive to the neurons.
Imagine the net excited to recurrent
so the net current flowing through all the excitatory synapses and
the net inhibitory current the net current flowing through all the inhibitory synapses
at the beginning of the simulation 
excitation is a little bit strong. So it outweighs the inhibition.
Overall these two combine and give and that potential that is above the firing threshold. In this situation
the neurons are firing regularly and
we are something randomly from the coincidence window
The net depression will drive the excitation down
such that
We get at one point where there's a balance between excitation and inhibition.
Their fluctuations are combining and only a positive fluctuation is reaching the firing threshold.
We're now in the fluctuation driven regime and spiking is irregular.
with stable point.
The plasticity rules are such that it won't go further down without being brought back up and
we stay in that irregular regime, so
to summarize
when we initialize the system
at a point where it's out weighed by excitation we get regular firing,
random something on the pre post timing on the
plasticity window,
net depression and
irregular firing an irregular firing that looks a lot like what we see from cells in-vivo
and
This is a surprisingly robust phenomenon, we can try the same thing
starting the simulation with different presynaptic excitatory rate.
- plot the
postsynaptic rate and
against the presynaptic
constant rate and we see the postsynaptic rate is scaling up but much weaker than the slope of one
So it's actually a very weak coupling.
This is accompanied most importantly by a CV that is very high at the end of the simulation and
doesn't depend on whether we start at a high precision optic rate or a low presynaptic rate.
This is due to a balance between excitation and inhibition.
By looking at the ratio of inhibition over excitation
So this is one very nice more homeostatic type of
function of this spike timing-dependent plasticity.
- the same
learning rule can allow neurons to pick up which of
the inputs are
correlated with the output.
- this is not the only way with which neuron are keeping into this
balance regime and there are other
mechanisms that can lead neurons for doing that for instance the plasticity of inhibitory cells.
- simulate a model of spike timing-dependent plasticity; observe the small leads to a self stabilization of the different types of
synapses that are impinging on the neuron that is the excitation is balancing the inhibition; observe how
the synaptic weights will depend on the level of correlation between the input and the output; 

## Model of STDP

The phenomenology of STDP is generally described as a biphasic exponentially decaying function. That is, the instantaneous change in weights is given by:

\begin{eqnarray}
& \Delta W &=& A_+ e^{ (t_{pre}-t_{post})/\tau_+}  & \text{if} \hspace{5mm}  t_{post} > t_{pre}& \\
& \Delta W &=& -A_- e^{- (t_{pre}-t_{post})/\tau_-}   &\text{if} \hspace{5mm} t_{post} < t_{pre}& \\
\end{eqnarray}

where $\Delta W$ denotes the change in the synaptic weight, $A_+$ and $A_-$ determine the maxmimum amount of synaptic modification (which occurs when the timing difference between presynaptic and postsynaptic spikes is close to zero), $\tau_+$ and $\tau_-$ determine the ranges of pre-to-postsynaptic interspike intervals over which synaptic strengthening or weakening occurs. Thus, $\Delta W > 0 $ means that postsynaptic neuron spikes after the presynaptic neuron.

This model captures the phenomena that repeated occurrences of presynaptic spikes within a few milliseconds **before** postsynaptic action potentials lead to long-term potentiation (LTP) of the synapse, whereas repeated occurrences of presynaptic spikes **after** the postsynaptic ones lead to long-term depression (LTD) of the same synapse.

The latency between presynaptic and postsynaptic spike ($\Delta t$) is defined as:

\begin{eqnarray}
\Delta t = t_{\rm pre} - t_{\rm post}
\end{eqnarray}

where $t_{\rm pre}$ and $t_{\rm post}$ are the timings of the presynaptic and postsynaptic spikes, respectively.


Complete the following code to set the STDP parameters and plot the STDP function. Note that for simplicity, we assume **$\tau_{+} = \tau_{-} = \tau_{\rm stdp}$**.

## Exercise 1: Compute the STDP changes $\Delta W$

Note, as shown above, the expression of $\Delta W$ is different for $t_{post}>t_{pre}$ and $t_{post}<t_{pre}$. In the code, we use the parameter `time_diff` that describes the $t_{pre}-t_{post}$, as given above.

After implementing the code, you can visualize the STDP kernel, which describes how much the synaptic weight will change given a latency between the presynaptic and postsynaptic spikes. 

In [ ]:
def Delta_W(pars, A_plus, A_minus, tau_stdp):
  """
  Plot STDP biphasic exponential decaying function

  Args:
    pars       : parameter dictionary
    A_plus     : (float) maxmimum amount of synaptic modification
                 which occurs when the timing difference between pre- and
                 post-synaptic spikes is positive
    A_plus     : (float) maxmimum amount of synaptic modification
                 which occurs when the timing difference between pre- and
                 post-synaptic spikes is negative
    tau_stdp   : the ranges of pre-to-postsynaptic interspike intervals
                 over which synaptic strengthening or weakening occurs

  Returns:
    dW         : instantaneous change in weights
  """
  #######################################################################
  ## TODO for students: compute dP, then remove the NotImplementedError #
  # Fill out when you finish the function
  raise NotImplementedError("Student excercise: compute dW, the change in weights!")
  #######################################################################
  # STDP change
  dW = np.zeros(len(time_diff))
  # Calculate dW for LTP
  dW[time_diff <= 0] = ...
  # Calculate dW for LTD
  dW[time_diff > 0] = ...

  return dW


pars = default_pars_STDP()
# Get parameters
A_plus, A_minus, tau_stdp = pars['A_plus'], pars['A_minus'], pars['tau_stdp']
# pre_spike time - post_spike time
time_diff = np.linspace(-5 * tau_stdp, 5 * tau_stdp, 50)

# Uncomment to test your function
# dW = Delta_W(pars, A_plus, A_minus, tau_stdp)
# mySTDP_plot(A_plus, A_minus, tau_stdp, time_diff, dW)

[*Click for solution*](https://github.com/NeuromatchAcademy/course-content/tree/master//tutorials/W3D1_RealNeurons/solutions/W3D1_Tutorial4_Solution_1ba13cde.py)

*Example output:*

<img alt='Solution hint' align='left' width=560 height=416 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/W3D1_RealNeurons/static/W3D1_Tutorial4_Solution_1ba13cde_0.png>



## Keeping track of pre- and postsynaptic spikes
Since a neuron will receive numerous presynaptic spike inputs, in order to implement STDP by taking into account different synapses, we first have to keep track of the pre- and postsynaptic spike times throughout the simulation. 

A convenient way to do this is to define the following equation for each postsynaptic neuron:

\begin{eqnarray}
\tau_{-} \frac{dM}{dt} = -M
\end{eqnarray}

and whenever the postsynaptic neuron spikes, 

\begin{eqnarray}
M(t) = M(t) - A_{-}
\end{eqnarray}

This way $M(t)$ tracks the number of postsynaptic spikes over the timescale $\tau_{-}$. 

Similarly, for each presynaptic neuron, we define:

\begin{eqnarray}
\tau_{+} \frac{dP}{dt} = -P
\end{eqnarray}

and whenever there is spike on the presynaptic neuron,

\begin{eqnarray}
P(t) = P(t) + A_{+}
\end{eqnarray}

The variables $M(t)$ and $P(t)$ are very similar to the equations for the synaptic conductances, i.e., $g_{i}(t)$, except that they are used to keep track of pre- and postsynaptic spike times on a much longer timescale. Note that, $M(t)$ is always negative, and $P(t)$ is always positive. You can probably already guess that $M$ is used to induce LTD and $P$ to induce LTP because they are updated by $A_{-}$ and $A_{+}$, respectively. 

**Important note:** $P(t)$ depends on the presynaptic spike times. If we know the presynaptic spike times, $P$ can be generated before simulating the postsynaptic neuron and the corresponding STDP weights.

## Visualization of $P$
Here, we will consider a scenario in which there is a single postsynaptic neuron connected to $N$ presynaptic neurons. 

For instance, we have one postsynaptic neuron which receives Poisson type spiking inputs from five presynaptic neurons. 

We can simulate $P$ for each one of the presynaptic neurons. 

## Exercise 2: Compute $dP$

Here, yet again, we use the Euler scheme, which has been introduced several times in the previous tutorials.

Similar to the dynamics of the membrane potential in the LIF model, in a time step $dt$, $P(t)$ will decrease by an amount of $\displaystyle{\frac{dt}{\tau_+}P(t)}$. Whereas, if a presynaptic spike arrives, $P(t)$ will instantaneously increase by an amount of $A_+$. Therefore, 

\\

\begin{eqnarray}
dP = -\displaystyle{\frac{dt}{\tau_+}P[t]} + A_+\cdot \text{sp_or_not}[t+dt]
\end{eqnarray}

\\

where the $\text{sp_or_not}$ is a binary variable, i.e., $\text{sp_or_not}=1$ if there is a spike within $dt$, and $\text{sp_or_not}=0$ otherwise.

In [ ]:
def generate_P(pars, pre_spike_train_ex):
  """
  track of pre-synaptic spikes

  Args:
    pars               : parameter dictionary
    pre_spike_train_ex : binary spike train input from
                         presynaptic excitatory neuron

  Returns:
    P                  : LTP ratio
  """

  # Get parameters
  A_plus, tau_stdp = pars['A_plus'], pars['tau_stdp']
  dt, range_t = pars['dt'], pars['range_t']
  Lt = range_t.size

  # Initialize
  P = np.zeros(pre_spike_train_ex.shape)
  for it in range(Lt - 1):
    #######################################################################
    ## TODO for students: compute dP, then remove the NotImplementedError #
    # Fill out when you finish the function
    raise NotImplementedError("Student excercise: compute P, the change of presynaptic spike")
    #######################################################################
    # Calculate the delta increment dP
    dP = ...
    # Update P
    P[:, it + 1] = P[:, it] + dP

  return P


# Uncomment these lines to test your function
# pars = default_pars_STDP(T=200., dt=1.)
# pre_spike_train_ex = Poisson_generator(pars, rate=10, n=5, myseed=2020)
# P = generate_P(pars, pre_spike_train_ex)
# my_example_P()

[*Click for solution*](https://github.com/NeuromatchAcademy/course-content/tree/master//tutorials/W3D1_RealNeurons/solutions/W3D1_Tutorial4_Solution_9de149a2.py)

*Example output:*

<img alt='Solution hint' align='left' width=485 height=413 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/W3D1_RealNeurons/static/W3D1_Tutorial4_Solution_9de149a2_0.png>



---
# Section 2: Implementation of STDP

Finally, to implement STDP in spiking networks, we will change the value of the peak synaptic conductance based on the presynaptic and postsynaptic timing, thus using the variables $P(t)$ and $M(t)$. 

Each synapse $i$ has its own peak synaptic conductance ($\bar g_i$), which may vary between $[0, \bar g_{max}]$, and will be modified depending on the presynaptic and postsynaptic timing. 

* When the $i_{th}$ presynaptic neuron elicits a spike, its corresponding peak conductance is updated according to the following equation:
  
  \\

  \begin{eqnarray}
  \bar g_i = \bar g_i + M(t)\bar g_{max}
  \end{eqnarray}

  \\

  Note that, $M(t)$ tracks the time since the last postsynaptic potential and is always negative. Hence, if *the postsynaptic neuron spikes shortly before the presynaptic neuron*, the above equation shows that the peak conductance will decrease. 

* When the postsynaptic neuron spikes, the peak conductance of **each** synapse is updated according to:

  \\

  \begin{eqnarray}
  \bar g_i = \bar g_i + P_i(t)\bar g_{max}, \forall i
  \end{eqnarray}

  \\

  Note that, $P_i(t)$ tracks the time since the last spike of $i_{th}$ pre-synaptic neuron and is always positive.

  Thus, the equation given above shows that if the presynaptic neuron spikes before the postsynaptic neuron, its peak conductance will increase.

### LIF neuron connected with synapses that show STDP 
In the following exercise, we connect $N$ presynaptic neurons to a single postsynaptic neuron. We do not need to simulate the dynamics of each presynaptic neuron as we are only concerned about their spike times. So, we will generate $N$ Poisson type spikes. Here, we will assume that all these inputs are excitatory. 

We need to simulate the dynamics of the postsynaptic neuron as we do not know its spike times. We model the postsynaptic neuron as an LIF neuron receiving only excitatory inputs.

\\

\begin{eqnarray}
\tau_m\frac{dV}{dt} = -(V-E_L) - g_E(t) (V(t)-E_E)\,
\end{eqnarray}

\\

where the total excitatory synaptic conductance  $g_{E}(t)$  is given by:

\\

\begin{eqnarray}
g_E(t) = \sum_{i=1}^{N} g_i(t) \,
\end{eqnarray}

\\

While simulating STDP, it is important to make sure that $\bar g_i$ never goes outside of its bounds. 


In [ ]:
# @title Function for LIF neuron with STDP synapses


def run_LIF_cond_STDP(pars, pre_spike_train_ex):
  """
  conductance-based LIF dynamics

  Args:
    pars               : parameter dictionary
    pre_spike_train_ex : spike train input from presynaptic excitatory neuron

  Returns:
    rec_spikes         : spike times
    rec_v              : mebrane potential
    gE                 : postsynaptic excitatory conductance
  """

  # Retrieve parameters
  V_th, V_reset = pars['V_th'], pars['V_reset']
  tau_m = pars['tau_m']
  V_init, V_L = pars['V_init'], pars['V_L']
  gE_bar, VE, tau_syn_E = pars['gE_bar'], pars['VE'], pars['tau_syn_E']
  gE_init = pars['gE_init']
  tref = pars['tref']
  A_minus, tau_stdp = pars['A_minus'], pars['tau_stdp']
  dt, range_t = pars['dt'], pars['range_t']
  Lt = range_t.size

  P = generate_P(pars, pre_spike_train_ex)

  # Initialize
  tr = 0.
  v = np.zeros(Lt)
  v[0] = V_init
  M = np.zeros(Lt)
  gE = np.zeros(Lt)
  gE_bar_update = np.zeros(pre_spike_train_ex.shape)
  gE_bar_update[:, 0] = gE_init  # note: gE_bar is the maximum value

  # simulation
  rec_spikes = []  # recording spike times
  for it in range(Lt - 1):
      if tr > 0:
          v[it] = V_reset
          tr = tr - 1
      elif v[it] >= V_th:   # reset voltage and record spike event
          rec_spikes.append(it)
          v[it] = V_reset
          M[it] = M[it] - A_minus
          gE_bar_update[:, it] = gE_bar_update[:, it] + P[:, it] * gE_bar
          id_temp = gE_bar_update[:, it] > gE_bar
          gE_bar_update[id_temp, it] = gE_bar
          tr = tref / dt

      # update the synaptic conductance
      M[it + 1] = M[it] - dt / tau_stdp * M[it]
      gE[it + 1] = gE[it] - (dt / tau_syn_E) * gE[it] + (gE_bar_update[:, it] * pre_spike_train_ex[:, it]).sum()
      gE_bar_update[:, it + 1] = gE_bar_update[:, it] + M[it]*pre_spike_train_ex[:, it]*gE_bar
      id_temp = gE_bar_update[:, it + 1] < 0
      gE_bar_update[id_temp, it + 1] = 0.

      # calculate the increment of the membrane potential
      dv = (-(v[it] - V_L) - gE[it + 1] * (v[it] - VE)) * (dt / tau_m)

      # update membrane potential
      v[it + 1] = v[it] + dv

  rec_spikes = np.array(rec_spikes) * dt

  return v, rec_spikes, gE, P, M, gE_bar_update

## Evolution of excitatory synaptic conductance
In the following exercise, we will simulate an LIF neuron receiving input from $N=300$ presynaptic neurons.

In [ ]:
pars = default_pars_STDP(T=200., dt=1.)  # Simulation duration 200 ms
pars['gE_bar'] = 0.024                   # max synaptic conductance
pars['gE_init'] = 0.024                  # initial synaptic conductance
pars['VE'] = 0.                          # [mV] Synapse reversal potential
pars['tau_syn_E'] = 5.                   # [ms] EPSP time constant

# generate Poisson type spike trains
pre_spike_train_ex = Poisson_generator(pars, rate=10, n=300, myseed=2020)
# simulate the LIF neuron and record the synaptic conductance
v, rec_spikes, gE, P, M, gE_bar_update = run_LIF_cond_STDP(pars,
                                                           pre_spike_train_ex)

In [ ]:
# @title Figures of the evolution of synaptic conductance

# @markdown Run this cell to see the figures!
plt.figure(figsize=(12., 8))
plt.subplot(321)
dt, range_t = pars['dt'], pars['range_t']
if rec_spikes.size:
  sp_num = (rec_spikes / dt).astype(int) - 1
  v[sp_num] += 10   # add artificial spikes
plt.plot(pars['range_t'], v, 'k')
plt.xlabel('Time (ms)')
plt.ylabel('V (mV)')

plt.subplot(322)
# Plot the sample presynaptic spike trains
my_raster_plot(pars['range_t'], pre_spike_train_ex, 10)

plt.subplot(323)
plt.plot(pars['range_t'], M, 'k')
plt.xlabel('Time (ms)')
plt.ylabel('M')

plt.subplot(324)
for i in range(10):
  plt.plot(pars['range_t'], P[i, :])
plt.xlabel('Time (ms)')
plt.ylabel('P')

plt.subplot(325)
for i in range(10):
  plt.plot(pars['range_t'], gE_bar_update[i, :])
plt.xlabel('Time (ms)')
plt.ylabel(r'$\bar g$')

plt.subplot(326)
plt.plot(pars['range_t'], gE, 'r')
plt.xlabel('Time (ms)')
plt.ylabel(r'$g_E$')

plt.tight_layout()
plt.show()

## Think!
* In the above, even though all the presynaptic neurons have the same average firing rate, many of the synapses seem to have been weakened? Did you expect that? 

* Total synaptic conductance is fluctuating over time. How do you expect $g_E$ to fluctuate if synapses did not show any STDP like behavior?

* Do synaptic weight ever reach a stationary state when synapses show STDP? 

*hint*:
1. think about the area under the curve; how are the synapses set? and variance? (think poisson)
2. how does weight influence spiking fluctuations?
3. Think about how the weight distribution changes?

[*Click for solution*](https://github.com/NeuromatchAcademy/course-content/tree/master//tutorials/W3D1_RealNeurons/solutions/W3D1_Tutorial4_Solution_07dba4f3.py)



## Distribution of synaptic weight
From the example given above, we get an idea that some synapses depotentiate, but what is the distribution of the synaptic weights when synapses show STDP? 

In fact, it is possible that even the synaptic weight distribution itself is a time-varying quantity. So, we would like to know how the distribution of synaptic weights evolves as a function of time. 

To get a better estimate of the weight distribution and its time evolution, we will increase the presynaptic firing rate to $15$Hz and simulate the postsynaptic neuron for $120$s. 

In [ ]:
# @title Functions for simulating a LIF neuron with STDP synapses


def example_LIF_STDP(inputrate=15., Tsim=120000.):
  """
  Simulation of a LIF model with STDP synapses

  Args:
    intputrate  :  The rate used for generate presynaptic spike trains
    Tsim        :  Total simulation time

  output:
    Interactive demo, Visualization of synaptic weights
  """

  pars = default_pars_STDP(T=Tsim,  dt=1.)
  pars['gE_bar'] = 0.024
  pars['gE_init'] = 0.014  # initial synaptic conductance
  pars['VE'] = 0.          # [mV]
  pars['tau_syn_E'] = 5.   # [ms]

  starttime = time.perf_counter()
  pre_spike_train_ex = Poisson_generator(pars, rate=inputrate, n=300,
                                         myseed=2020)  # generate Poisson trains
  v, rec_spikes, gE, P, M, gE_bar_update = run_LIF_cond_STDP(pars,
                                                             pre_spike_train_ex)  # simulate LIF neuron with STDP
  gbar_norm = gE_bar_update/pars['gE_bar']  # calculate the ratio of the synaptic conductance
  endtime = time.perf_counter()
  timecost = (endtime - starttime) / 60.

  print('Total simulation time is %.3f min' % timecost)

  my_layout.width = '620px'

  @widgets.interact(
      sample_time=widgets.FloatSlider(0.5, min=0., max=1., step=0.1,
                                      layout=my_layout)
  )


  def my_visual_STDP_distribution(sample_time=0.0):
    sample_time = int(sample_time * pars['range_t'].size) - 1
    sample_time = sample_time * (sample_time > 0)

    plt.figure(figsize=(8, 8))
    ax1 = plt.subplot(211)
    for i in range(50):
      ax1.plot(pars['range_t'][::1000] / 1000., gE_bar_update[i, ::1000], lw=1., alpha=0.7)

    ax1.axvline(1e-3 * pars['range_t'][sample_time], 0., 1., color='k', ls='--')
    ax1.set_ylim(0, 0.025)
    ax1.set_xlim(-2, 122)
    ax1.set_xlabel('Time (s)')
    ax1.set_ylabel(r'$\bar{g}$')

    bins = np.arange(-.05, 1.05, .05)
    g_dis, _ = np.histogram(gbar_norm[:, sample_time], bins)
    ax2 = plt.subplot(212)
    ax2.bar(bins[1:], g_dis, color='b', alpha=0.5, width=0.05)
    ax2.set_xlim(-0.1, 1.1)
    ax2.set_xlabel(r'$\bar{g}/g_{\mathrm{max}}$')
    ax2.set_ylabel('Number')
    ax2.set_title(('Time = %.1f s' % (1e-3 * pars['range_t'][sample_time])),
                  fontweight='bold')
    plt.show()


print(help(example_LIF_STDP))

### Interactive Demo: Example of an LIF model with STDP

In [ ]:
# @title

# @markdown Make sure you execute this cell to enable the widget!

example_LIF_STDP(inputrate=15)

## Think!
Increase the firing rate (i.e., 30 Hz) of presynaptic neurons, and investigate the effect on the dynamics of synaptic weight distribution.

*hint*:
How do synapses move on increasing the firing rate? What if firing rate is increased in early times?

[*Click for solution*](https://github.com/NeuromatchAcademy/course-content/tree/master//tutorials/W3D1_RealNeurons/solutions/W3D1_Tutorial4_Solution_fe78e137.py)



# Section 3: Effect of input correlations

Thus far, we assumed that the input population was uncorrelated. What do you think will happen if presynaptic neurons were correlated? 

In the following, we will modify the input such that first $L$ neurons have identical spike trains while the remaining inputs are uncorrelated. This is a highly simplified model of introducing correlations. You can try to code your own model of correlated spike trains. 

In [ ]:
#@title Function for LIF neuron with STDP synapses receiving correlated inputs


def example_LIF_STDP_corrInput(inputrate=20., Tsim=120000.):
  """
  A LIF model equipped with STDP synapses, receiving correlated inputs

  Args:
    intputrate      :  The rate used for generate presynaptic spike trains
    Tsim            :  Total simulation time

  Returns:
    Interactive demo: Visualization of synaptic weights
  """

  np.random.seed(2020)
  pars = default_pars_STDP(T=Tsim, dt=1.)
  pars['gE_bar'] = 0.024
  pars['VE'] = 0.                                # [mV]
  pars['gE_init'] = 0.024 * np.random.rand(300)  # initial synaptic conductance
  pars['tau_syn_E'] = 5.                         # [ms]

  starttime = time.perf_counter()
  pre_spike_train_ex = Poisson_generator(pars, rate=inputrate, n=300, myseed=2020)
  for i_pre in range(50):
    pre_spike_train_ex[i_pre, :] = pre_spike_train_ex[0, :]  # simple way to set input correlated
  v, rec_spikes, gE, P, M, gE_bar_update = run_LIF_cond_STDP(pars, pre_spike_train_ex) # simulate LIF neuron with STDP
  gbar_norm = gE_bar_update / pars['gE_bar']  # calculate the ratio of the synaptic conductance
  endtime = time.perf_counter()
  timecost = (endtime - starttime) / 60.

  print(f'Total simulation time is {timecost:.3f} min')

  my_layout.width = '620px'

  @widgets.interact(
      sample_time=widgets.FloatSlider(0.5, min=0., max=1., step=0.05,
                                      layout=my_layout)
  )


  def my_visual_STDP_distribution(sample_time=0.0):

    sample_time = int(sample_time * pars['range_t'].size) - 1
    sample_time = sample_time*(sample_time > 0)
    figtemp = plt.figure(figsize=(8, 8))
    ax1 = plt.subplot(211)
    for i in range(50):
      ax1.plot(pars['range_t'][::1000] / 1000.,
               gE_bar_update[i, ::1000], lw=1., color='r', alpha=0.7, zorder=2)

    for i in range(50):
      ax1.plot(pars['range_t'][::1000] / 1000.,
               gE_bar_update[i + 50, ::1000], lw=1., color='b',
               alpha=0.5, zorder=1)

    ax1.axvline(1e-3 * pars['range_t'][sample_time], 0., 1.,
                color='k', ls='--', zorder=2)
    ax1.set_ylim(0, 0.025)
    ax1.set_xlim(-2, 122)
    ax1.set_xlabel('Time (s)')
    ax1.set_ylabel(r'$\bar{g}$')
    legend=ax1.legend(['Correlated input', 'Uncorrelated iput'], fontsize=18,
                      loc=[0.92, -0.6], frameon=False)
    for color, text, item in zip(['r', 'b'], legend.get_texts(), legend.legendHandles):
      text.set_color(color)
      item.set_visible(False)

    bins = np.arange(-.05, 1.05, .05)
    g_dis_cc, _ = np.histogram(gbar_norm[:50, sample_time], bins)
    g_dis_dp, _ = np.histogram(gbar_norm[50:, sample_time], bins)

    ax2 = plt.subplot(212)
    ax2.bar(bins[1:], g_dis_cc, color='r', alpha=0.5, width=0.05)
    ax2.bar(bins[1:], g_dis_dp, color='b', alpha=0.5, width=0.05)
    ax2.set_xlim(-0.1, 1.1)
    ax2.set_xlabel(r'$\bar{g}/g_{\mathrm{max}}$')
    ax2.set_ylabel('Number')
    ax2.set_title(('Time = %.1f s' % (1e-3 * pars['range_t'][sample_time])),
                  fontweight='bold')
    plt.show()


print(help(example_LIF_STDP_corrInput))

## Interactive Demo: LIF model with plastic synapses receiving correlated inputs

In [ ]:
# @title

# @markdown Make sure you execute this cell to enable the widget!

example_LIF_STDP_corrInput(inputrate=10.0)

**Why do weights of uncorrelated neurons decrease when synapses show STDP**

Above, we notice that the synapses of correlated neurons (which spike together) were almost unaffected, but the weights of other neurons diminished. Why does this happen? 

The reason is that the correlated presynaptic neurons have a higher chance of eliciting a spike in the postsynaptic neurons and that create a $pre \rightarrow post$ pairing of spikes. 

## Think!

* Modify the code above and create two groups of correlated presynaptic neurons and test what happens to the weight distribution.

* How can the above observations be used to create unsupervised learning? Could you imagine how we have to train a neuronal model enabled with STDP rule to identify input patterns?

* What else can be done with this type of plasticity?


*hint*:
1. The two groups compete for?
2. Think in terms of correlations. 
3. Think of circuit formations. What's the simplest circuit that can be formed?

[*Click for solution*](https://github.com/NeuromatchAcademy/course-content/tree/master//tutorials/W3D1_RealNeurons/solutions/W3D1_Tutorial4_Solution_b397ae39.py)



---
# Summary

Hooray! You have just finished this loooong day! In this tutorial, we covered the concept of **spike-timing dependent plasticity (STDP)**.

We managed to:

- build a model of synapse that shows STDP.

- study how correlations in input spike trains influence the distribution of synaptic weights.

Using presynaptic inputs as Poisson type spike trains, we modeled an LIF model with synapses equipped with STDP. We also studied the effect of correlated inputs on the synaptic strength!